Code overview for StyleBot:

1. Importing Libraries: 
The code begins by importing necessary libraries such as tkinter for GUI, requests for making HTTP requests, json for handling JSON data, datetime for working with timestamps, PIL for image processing, and torch for deep learning functionalities.

2. Model Definition: 
The Generator class defines a neural network model used for generating images. It's implemented using PyTorch and consists of several convolutional layers.

3. Loading Pre-trained Model: 
The pre-trained generator model is loaded from a file ('generator.pth').

4. User Interface: 
The tkinter library is used to create a graphical user interface (GUI) for the application. It includes tabs for multiple chat sessions, a text entry for user input, and buttons for adding tabs, closing tabs, adding images, and sending messages.

5. Chat Functionality: 
The chat function sends user messages to a local server and receives responses. It handles the communication with the server using HTTP requests.

6. Processing Images: 
Functions are defined to process and display images within the GUI. Images can be uploaded by the user and displayed alongside the chat messages.

7. Message Handling: 
User messages are sent to the chat function, and responses are displayed in the chat area. If the user input contains shopping-related keywords, the system generates and displays fake shopping images.

8. Button Actions: 
Various buttons in the GUI have associated actions. These include adding new tabs, closing tabs, adding images, and sending messages.

9. Main Loop: 
The main loop (root.mainloop()) runs the tkinter application, keeping it active and responsive to user interactions.

10. Additional Notes:
The code utilizes tags ('user_message' and 'bot_message') for styling user and bot messages differently in the chat area.
Error handling is minimal; exceptions raised during HTTP requests are not thoroughly managed.
The code assumes the presence of a local server running on port 11434 for handling chat interactions.

In [5]:
import tkinter as tk
from tkinter import scrolledtext, ttk, filedialog
import requests
import json
from datetime import datetime
from PIL import Image, ImageTk
import torch
import torchvision.transforms as transforms

model = "llama3"

shopping_keywords = ['black', 'white', 'red', 'blue', 'green', 'yellow', 'purple', 'pink', 'grey', 'beige',
                     'stripes', 'floral', 'patterned', 'solid', 'animal print', 'geometric',
                     'sleeves', 'long sleeves', 'short sleeves', 'sleeveless', 'straps', 'spaghetti straps', 'no straps',
                     'neckline', 'v-neck', 'halter', 'keyhole', 'length', 'long', 'short', 'maxi', 'fit', 'fitted', 'loose', 'flowy',
                     'cotton', 'silk', 'denim', 'casual', 'formal', 'sporty']

current_image_path = None  
current_image = None  

# Load the Generator model
class Generator(torch.nn.Module):
    def __init__(self, latent_size, num_channels=3):
        super(Generator, self).__init__()
        self.latent_size = latent_size
        self.main = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(latent_size, 1024, 4, 1, 0, bias=False),
            torch.nn.BatchNorm2d(1024),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(128, num_channels, 4, 2, 1, bias=False),
            torch.nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

# Load the generator model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(latent_size=100).to(device)
generator.load_state_dict(torch.load('generator.pth', map_location=device))
generator.eval()

def generate_image(latent_size=100):
    with torch.no_grad():
        noise = torch.randn(1, latent_size, 1, 1, device=device)
        fake_image = generator(noise).detach().cpu()
    return fake_image

def format_message(sender, content, color):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    return f"{timestamp} - {sender}: {content}\n", color

def chat(messages):
    r = requests.post(
        "http://localhost:11434/api/chat",
        json={"model": model, "messages": messages, "stream": True},
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
        if body.get("done", False):
            message["content"] = output
            return message

def close_tab():
    selected_tab = tab_control.index("current")
    tab_control.forget(selected_tab)

def process_image(image, chat_area):
    if isinstance(image, Image.Image):
        pass
    elif isinstance(image, str):
        image = Image.open(image)
    else:
        raise ValueError("Invalid image input")

    max_width = 400  
    if image.width > max_width:
        ratio = max_width / image.width
        new_height = int(image.height * ratio)
        image = image.resize((max_width, new_height), Image.LANCZOS) 

    global current_image
    current_image = ImageTk.PhotoImage(image)

    chat_area.config(state=tk.NORMAL)
    chat_area.image_create(tk.END, image=current_image)
    chat_area.insert(tk.END, '\n\n')
    chat_area.config(state=tk.DISABLED)

def add_image():
    global current_image_path 
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif;*.bmp")])
    if file_path:
        current_image_path = file_path  
        selected_tab = tab_control.nametowidget(tab_control.select())
        chat_area = selected_tab.chat_area
        chat_area.config(state=tk.NORMAL)
        chat_area.insert(tk.END, f"Image: {file_path}\n", 'user_message')
        process_image(file_path, chat_area)
        chat_area.config(state=tk.DISABLED)

def send_message():
    global current_image_path  
    user_input = user_input_entry.get()
    if user_input:
        selected_tab = tab_control.nametowidget(tab_control.select())
        chat_area = selected_tab.chat_area
        chat_area.config(state=tk.NORMAL)
        
        if current_image_path:
            chat_area.insert(tk.END, f"You: Image\n", 'user_message')
            process_image(current_image_path, chat_area)
            chat_area.insert(tk.END, f"Image: {current_image_path}\n", 'user_message')
            current_image_path = None
        else:
            chat_area.insert(tk.END, f"You: {user_input}\n", 'user_message')
            if is_shopping_question(user_input):
                display_shopping_images(user_input, shopping_keywords, chat_area)
            else:
                message = chat([{"role": "user", "content": user_input}])
                bot_response = message.get("content", "")
                chat_area.insert(tk.END, "Bot: ", 'bot_message')
                chat_area.insert(tk.END, f"{bot_response}\n", 'bot_message')
        chat_area.config(state=tk.DISABLED)
        user_input_entry.delete(0, tk.END)

def is_shopping_question(user_input):
    return any(keyword in user_input.lower() for keyword in shopping_keywords)

def display_shopping_images(user_input, shopping_keywords, chat_area):
    for i in range(3):
        fake_image_tensor = generate_image()
        fake_image = transforms.ToPILImage()(fake_image_tensor[0])
        process_image(fake_image, chat_area)

root = tk.Tk()
root.title("StyleBot")

root.configure(background="#f0f0f0")
root.geometry("800x600")

tab_control = ttk.Notebook(root)
tab_control.pack(fill='both', expand='yes')

def add_new_tab():
    new_tab_name = f"Tab {tab_control.index('end') + 1}"
    new_chat_tab = ttk.Frame(tab_control)
    new_chat_area = scrolledtext.ScrolledText(new_chat_tab, width=60, height=25, wrap=tk.WORD, font=("Helvetica", 12),
                                              bg="white", relief=tk.SOLID, bd=1)
    new_chat_area.pack(padx=10, pady=10, side=tk.TOP)
    tab_control.add(new_chat_tab, text=new_tab_name)
    new_chat_tab.chat_area = new_chat_area
    new_chat_area.tag_configure('user_message', foreground='blue')
    new_chat_area.tag_configure('bot_message', foreground='green')

add_new_tab()

user_input_entry = tk.Entry(root, width=60, font=("Helvetica", 12), relief=tk.FLAT, bd=1)
user_input_entry.pack(padx=10, pady=5, side=tk.TOP)

button_frame = tk.Frame(root, bg="#f0f0f0")
button_frame.pack(side=tk.BOTTOM, fill=tk.X)

add_tab_button = tk.Button(button_frame, text="Add Tab", command=add_new_tab, bg="#4CAF50", fg="white", font=("Helvetica", 12), relief=tk.FLAT, bd=1)
add_tab_button.pack(padx=10, pady=5, side=tk.LEFT)

close_tab_button = tk.Button(button_frame, text="Close Tab", command=close_tab, bg="#FF5722", fg="white", font=("Helvetica", 12), relief=tk.FLAT, bd=1)
close_tab_button.pack(padx=10, pady=5, side=tk.LEFT)

add_image_button = tk.Button(button_frame, text="Add Image", command=add_image, bg="#2196F3", fg="white", font=("Helvetica", 12), relief=tk.FLAT, bd=1)
add_image_button.pack(padx=10, pady=5, side=tk.RIGHT)

send_button = tk.Button(button_frame, text="Send", command=send_message, bg="#4CAF50", fg="white", font=("Helvetica", 12), relief=tk.FLAT, bd=1)
send_button.pack(padx=10, pady=5, side=tk.RIGHT)

root.mainloop()